# Tutorial 06 - Working with Electrical Stimulation Data

In addition to pyeCAP.Ephys for loading in electrophysiology data, pyeCAP also has a class, pyeCAP.Stim, for working with stimulation data from different vendors in a consistent format. This tutorial covers the basics of loading in and working with stimulation data that is paired with an electrophysiology data set.

First, we will load in an electrophysiology data set.

In [1]:
# import statements
import pyeCAP
import matplotlib.pyplot as plt

# put matplotlib in widget mode so that we can use interactive plots
%matplotlib widget 

# path to directory containing the TDT tank
directory = r"pnpig191126-191204-174838"
data = pyeCAP.Ephys(directory, stores=["RawE", "RawG"])

# Starting by removing the excess channel 'RawG 4'. We could do this either by the index in the channel list (7) or using the channel name directly. 
data = data.remove_ch('RawG 4')

# We can rename the channels using the set_ch_names method and providing a list of new channel names that matches the number of channels in the data set.
data = data.set_ch_names(['LIFE 1', 'LIFE 2', 'LIFE 3', 'LIFE 4', 'EMG 1', 'EMG 2', 'EMG 3'])

# Lets also set the types of the individual channels so that we can indepentently interact with LIFE and EMG channels
data = data.set_ch_types(['LIFE']*4 + ['EMG']*3)

We can load in a stimulation data set in two ways. The first is by passing a file path for the data set contining stimulation infomation. For TDT data sets, stimulation data is stored in the same tdt tank as electrophysiology data and we can pass the same directory path to pyeCAP.Stim as we passed to pyeCAP.Ephys.

In [2]:
stim_data = pyeCAP.Stim(directory)

ValueError: No electrical stimulation detected

***TODO: Right now loading stim data directly from any other previously loaded pyeCAP object does not work as expected. For now data must be loaded from a file path or list of files.***

The second way that we could load in a stimulation data set that corresponds to an already loaded pyeCAP object, such as a pyeCAP.Ephys object, is to pass the pyeCAP object to the pyeCAP.Stim constructor. In this case, *data* is a pyeCAP.Ephys object and we could have initialized pyeCAP.Stim with the following code.

        stim_data = pyeCAP.Stim(data)

## Exploration of Stimulation Data

When you use pyeCAP to read in electrical stimulation data, it puts data from different input data types into a consistent format that is easy to interact with.

Let's start by looking at the the paramters that were applied during this experiment.

In [3]:
stim_data.parameters

NameError: name 'stim_data' is not defined

This returns a human readable table of stimulation parameters. For people familiar with the standard scientific computing libraries in python, stim_data.parameters return a pandas DataFrame. Useful infromation on working with pandas DataFrames can be found here: https://pandas.pydata.org/docs/getting_started/intro_tutorials/03_subset_data.html#min-tut-03-subset. 

In pyeCAP we generally work with stimulation data by referencing the index of a particular stimulation within the stim_data.parameters table. Using the indexing methods available through pandas makes it easy to obtain and work with a subset of stimulation parameters, work with parameters sorted on a particular value(s), etc. You will see this in more detail within both this tutorial and Tutorial 7 on Evoked Compound Action Potential analysis. 

We can easily plot our stimulation data alongside the relevent electrophysiology data using the same plot method we have seen in previous tutorials. This is a nice way to interactively explore data and view the effects of stimulation. To do this, we simply have to pass stim_data to the events attribute of the plot method.

In [ ]:
%matplotlib widget
data.plot(x_lim=(0, 2), events=stim_data, show='notebook')

Now you can see stimulation events on the scrollbar axis of the plot (grey bars) as well as a new event axis at the top of the plot showing the both the stimuation events (grey pulses) but also the individual event data for stimulation pulses (orange line). Scroll through the data set to see both of these.

## Working with Multiple Stimulation Data Sets

Similar to electrophysiology data multiple stimulation data sets can be read in and worked with simultaneously. To show this, let's use our same code from Tutorial 03 on working with multiple data sets to read in a list of TDT data sets. 

In [ ]:
tdt_tanks = [r"C:\Users\malal\Documents\WITNE\pyCAP\TutorialExamples\pnpig191126-191204-151801", r"C:\Users\malal\Documents\WITNE\pyCAP\TutorialExamples\pnpig191126-191204-152107"]

Now we can easily load in both the electrophysiology data and the stimulation data from this list of directories.

In [3]:
m_data = pyeCAP.Ephys(tdt_tanks, stores=["RawE", "RawG"])
m_stim_data = pyeCAP.Stim(tdt_tanks)

NameError: name 'tdt_tanks' is not defined

If we take a look at the parameter list for *m_stim_data*, we now have 10 groups of stimulation paramters corresponding to the 10 data sets we loaded into the pyeCAP.Ephys and pyeCAP.Stim data sets.

In [4]:
m_stim_data.parameters

NameError: name 'm_stim_data' is not defined